# 14.4 양방향 LSTM과 어텐션 메커니즘(BiLSTM with Attention mechanism)

단뱡항 LSTM으로 텍스트 분류를 수행할 수도 있지만 때로는 양방향 LSTM을 사용하는 것이 더 강력합니다. 여기에 추가적으로 어텐션 메커니즘을 사용할 수도 있습니다. 양방향 LSTM과 어텐션 메커니즘으로 IMDB 리뷰 감성 분류하기를 수행해봅시다.

### 1. IMDB 리뷰 데이터 전처리하기

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

IMDB 리뷰 데이터는 앞서 텍스트 분류하기 챕터에서 다룬 바 있으므로 데이터에 대한 상세 설명은 생략합니다. 최대 단어 개수를 10,000으로 제한하고 훈련 데이터와 테스트 데이터를 받아옵니다.

In [2]:
import numpy as np
# np version 
old = np.load
np.load = lambda *a, **k: old(*a, allow_pickle=True, **k)

vc_size = 10_000
(x_train,y_train), (x_test, y_test) = imdb.load_data(num_words=vc_size)

#
np.load = old
del(old)
#

훈련 데이터와 이에 대한 레이블이 각각 X_train, y_train에 테스트 데이터와 이에 대한 레이블이 각각 X_test, y_test에 저장되었습니다. IMDB 리뷰 데이터는 이미 정수 인코딩이 된 상태므로 남은 전처리는 패딩뿐입니다. 리뷰의 최대 길이와 평균 길이를 확인해봅시다.

In [3]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in x_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, x_train))/len(x_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


리뷰의 최대 길이는 2,494이며 리뷰의 평균 길이는 약 238로 확인됩니다. 평균 길이보다는 조금 크게 데이터를 패딩하겠습니다.

In [4]:
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

훈련용 리뷰와 테스트용 리뷰의 길이가 둘 다 500이 되었습니다. 이번에는 이진 분류를 위해 소프트맥스 함수를 사용할 것이므로, y_train과 y_test 모두 원-핫 인코딩을 해줍니다.

In [5]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### 2. 바다나우 어텐션(Bahdanau Attention)

여기서 사용할 어텐션은 바다나우 어텐션(Bahdanau attention)입니다. 이를 이해하기 위해 앞서 배운 가장 쉬운 어텐션이었던 닷 프로덕트 어텐션과 어텐션 스코어 함수의 정의를 상기해봅시다.

어텐션 스코어 함수란 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수를 말합니다. 그리고 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)이었습니다. 다음은 닷 프로덕트 어텐션의 어텐션 스코어 함수를 보여줍니다.

$core(query, key)=query^key$

바다나우 어텐션은 아래와 같은 어텐션 스코어 함수를 사용합니다.

$score(query, key)=V^Ttanh(W_1key+W_2query)$

이 어텐션 스코어 함수를 사용하여 어텐션 메커니즘을 구현하면 됩니다. 그런데 텍스트 분류에서 어텐션 메커니즘을 사용하는 이유는 무엇일까요? RNN의 마지막 은닉 상태는 예측을 위해 사용됩니다. 그런데 이 RNN의 마지막 은닉 상태는 몇 가지 유용한 정보들을 손실한 상태입니다. 그래서 RNN이 time step을 지나며 손실했던 정보들을 다시 참고하고자 합니다.

이는 다시 말해 RNN의 모든 은닉 상태들을 다시 한 번 참고하겠다는 것입니다. 그리고 이를 위해서 어텐션 메커니즘을 사용합니다.

In [44]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [45]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Concatenate, BatchNormalization
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

이제 모델을 설계해보겠습니다. 여기서는 케라스의 함수형 API를 사용합니다. 우선 입력층과 임베딩층을 설계합니다.

In [46]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vc_size, 128, input_length=max_len)(sequence_input)

10,000개의 단어들을 128차원의 임베딩 벡터로 임베딩하도록 설계하였습니다. 이제 양방향 LSTM을 설계합니다.

In [47]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
    ( LSTM
        ( 128
        ,  dropout=0.3
        , return_sequences=True
        , return_state = True
        , recurrent_activation='relu'
        , recurrent_initializer='glorot_uniform'))(embedded_sequences)

양방향 LSTM을 설계하였습니다. 각 상태의 크기(shape)를 출력해보겠습니다.

In [48]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 256) (None, 128) (None, 128) (None, 128) (None, 128)


순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장하고, 역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장합니다.

각 은닉 상태나 셀 상태의 경우에는 128차원을 가지는데, lstm의 경우에는 (500 × 256)의 크기를 가집니다. foward 방향과 backward 방향이 연결된 hidden state벡터가 모든 시점에 대해서 존재함을 의미합니다.

양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

In [49]:
state_h = Concatenate()([forward_h, backward_h]) # 은익 상태
state_c = Concatenate()([forward_c, backward_c]) # Shell state

어텐션 메커니즘에서는 은닉 상태를 사용합니다. 이를 입력으로 컨텍스트 벡터(context vector)를 얻습니다.

In [52]:
attention = BahdanauAttention(128)  # Define weight size
context_vector, attention_weights = attention(lstm, state_h)

In [53]:
hidden = BatchNormalization()(context_vector)

이진 분류이므로 출력층에 2개의 뉴런을 배치하고, 활성화 함수로 소프트맥스 함수를 사용합니다.

In [54]:
output = Dense(2, activation='softmax')(hidden)
model = Model(inputs=sequence_input, outputs=output)

옵티마이저로 아담 옵티마이저를 정의합니다. tf.keras.opimizers에 있는 옵티마이저들은 그래디언트 클리핑을 위한 두 개의 매개변수를 제공합니다. 하나는 L2 노름의 임계값을 지정하는 clipnorm이고 다른 하나는 절대값으로 임계값을 지정하는 clipvalue입니다.

clipnorm 매개변수가 설정되면 그래디언트의 L2 노름이 clipnorm보다 클 경우 다음과 같이 클리핑된 그래디언드를 계산합니다.

클리핑된 그래디언트 = 그래디언트 * clipnorm / 그래디언트의 L2 노름

clipvalue 매개변수가 설정되면 -clipvalue보다 작은 그래디언트는 -clipvalue가 되고 clipvalue보다 큰 그래디언트는 clipvalue로 만듭니다. 이 두 클리핑 방식을 동시에 사용할 수도 있습니다. 여기서는 clipnorm 매개변수만 지정했습니다.

In [55]:
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)

정의된 옵티마이저를 사용하여 모델을 컴파일합니다.

In [58]:
model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

소프트맥스 함수를 사용하므로 손실 함수로 categorical_crossentropy를 사용하였습니다. 이제 모델을 훈련하겠습니다.

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose =1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 2367s 95ms/sample - loss: 0.6821 - accuracy: 0.5594 - val_loss: 0.6848 - val_accuracy: 0.6766
Epoch 2/10
25000/25000 [==============================] - 2639s 106ms/sample - loss: 0.5778 - accuracy: 0.7075 - val_loss: 0.5621 - val_accuracy: 0.7826
Epoch 3/10
25000/25000 [==============================] - 3017s 121ms/sample - loss: 0.4733 - accuracy: 0.7763 - val_loss: 0.4204 - val_accuracy: 0.8274
Epoch 4/10
25000/25000 [==============================] - 2653s 106ms/sample - loss: 0.4016 - accuracy: 0.8225 - val_loss: 0.3611 - val_accuracy: 0.8457
Epoch 5/10
20480/25000 [=======================>......] - ETA: 7:06 - loss: 0.3521 - accuracy: 0.8479

검증 데이터로 테스트 데이터를 사용하여 에포크가 끝날 때마다 테스트 데이터에 대한 정확도를 출력하도록 하였습니다.

In [ ]:
print('\n Test accuracy is : %.4f' % model.evaluate(x_test, y_test)[1]))